In [1]:
import os
import json
import random
import traceback

from datetime import datetime

from tqdm.notebook import tqdm

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.fact_score.fact_scorer import FactScorer
from experiments.metrics.bert_score import BertScore
from experiments.metrics.llm_judge import LLMAsJudgeEvaluator

In [2]:
USE_MCP = False
DEBUG_MODE = False
USE_TRIAGE = False

LLM_JUDGE_MODEL = "gemini-2.0-flash"
# LLM_JUDGE_MODEL = "gpt-4o"

fact_scorer = FactScorer()
path_to_source_data = "./resources/02_facts/"

date_str = datetime.now().strftime("%d%m%Y")
path_to_results = f"./results/{date_str}"

if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

time_str = datetime.now().strftime("%H%M%S")

result_file_name = f"{path_to_results}/{time_str}_results.json"
failed_experiments_file_name = f"{path_to_results}/{time_str}_failed_experiments.json"

files = {
    "extensibility": "extensibility_assistance_facts.json",
    "malaysia": "malaysia_support_facts.json",
    "peppol": "peppol_support_facts.json",
    "all": "all_cases_facts.json",
}

In [3]:
file_path = os.path.abspath(path_to_source_data + files["all"])

with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [4]:
random_data_set = random.sample(data_set, 20)

In [5]:
triage_agent = Triage()
execution_trail = ""
experiment_results: list[ExperimentResult] = []
failed_experiments: list = []

for data_row in tqdm(random_data_set, desc="Running experiments", unit="experiment"):
    try:
        result = ExperimentResult(
            id=data_row.id,
            question=data_row.question,
            answer=data_row.answer,
            product=data_row.product,
            category=data_row.category,
            persona=data_row.persona,
            activity=data_row.activity,
            country=data_row.country,
        )
        print(f"Asking agent question with id: {data_row.id}")
        if USE_TRIAGE:
            print("\tUsing Triage to categorize the query...")
            query_categorization = triage_agent.triage_user_message(
                user_message=data_row.question
            )
            if DEBUG_MODE:
                print(f"\tQuery categorization: {query_categorization}")

        result.triage_category = (
            query_categorization["category"]
            if USE_TRIAGE
            else TriageSettings.Categories.ALL
        )

        tools = ToolsFabric.get_tools_for_category(
            use_mcp=USE_MCP,
            configuration=result.triage_category,
        )

        if USE_MCP:
            async with MultiServerMCPClient(tools) as client:
                agent = ReActAgent(tool_list=client.get_tools())
                execution_trail = await agent.arun_agent_with_input(
                    user_message=data_row.question, debug=DEBUG_MODE
                )
        else:
            agent = ReActAgent(tool_list=tools)
            execution_trail = agent.run_agent_with_input(
                user_message=data_row.question, debug=DEBUG_MODE
            )

        run_data = agent.get_execution_data()

        result.tools_used = run_data.tools_used
        result.tool_calls_count = len(run_data.tools_used)
        result.excecution_time_seconds = run_data.excecution_time_seconds
        result.model_used = run_data.model_used
        result.tokens_consumed = run_data.tokens_consumed
        result.llm_call_count = run_data.llm_call_count
        result.facts = data_row.facts
        result.generated_answer = run_data.final_output

        print("\tFinished agent execution")

        print("\tCalculating FactScore...")
        result.fact_score = await fact_scorer.get_fact_score(
            facts=data_row.facts,
            knowledge_source=result.generated_answer,
            debug=DEBUG_MODE,
        )

        print("\tCalculating BERTScore...")
        result.bert_score = BertScore.compute_score(
            expected_response=data_row.answer, actual_response=result.generated_answer
        )

        print("\tEvaluating agent response with LLM as a Judge...")
        result.llm_judge_model = LLM_JUDGE_MODEL
        llm_evaluator = LLMAsJudgeEvaluator(model=LLM_JUDGE_MODEL)

        result.llm_judge_outcome = llm_evaluator.evaluate(
            question=data_row.question, generated_answer=result.generated_answer
        )

        result.llm_judge_call_count = llm_evaluator.get_llm_call_count()
        result.llm_judge_tokens_consumed = llm_evaluator.get_token_consumption()

        print(f"\tFinished experiment for id: {data_row.id}\n")

        experiment_results.append(result)
    except Exception as e:
        print(f"\tExperiment failed for id {data_row.id}: {e}")
        if DEBUG_MODE:
            traceback.print_exc()

        failed_experiments.append(
            {
                "id": data_row.id,
                "error": str(e),
                "traceback": traceback.format_exc(),
            }
        )

Running experiments:   0%|          | 0/20 [00:00<?, ?experiment/s]

Asking agent question with id: EA-50
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...
	Finished experiment for id: EA-50

Asking agent question with id: EA-21
	Finished agent execution
	Calculating FactScore...
	Experiment failed for id EA-21: Error while parsing the response of the LLM
Asking agent question with id: PS-8
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...
	Finished experiment for id: PS-8

Asking agent question with id: EA-41
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...
	Finished experiment for id: EA-41

Asking agent question with id: PS-5
	Finished agent execution
	Calculating FactScore...
	Calculating BERTScore...
	Evaluating agent response with LLM as a Judge...
	Finished experiment for id: PS-5

Asking agent question with id: PS-

In [6]:
if failed_experiments:
    with open(failed_experiments_file_name, "w") as f:
        json.dump(failed_experiments, f, indent=2)

In [7]:
# Serialize experiment results to list of dicts
import pandas as pd

records = [r.model_dump(mode="json") for r in experiment_results]
# Make it a dataframe in order to flatten nested structures
df = pd.json_normalize(records)

df

/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...med function module.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(direct_fa...upporting this fact.')])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/i589287/Library/Caches/pypoetry/virtualenvs/react-agent-CUxtcVGL-py3.13/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `FactScoreResult` with value `FactScoreResult(dire

,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,llm_judge_tokens_consumed.input_tokens,llm_judge_tokens_consumed.output_tokens,llm_judge_tokens_consumed.total_tokens,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,EA-50,What are the procedures for extending an eDocu...,The procedures for extending an eDocument Acti...,None,None,None,None,None,[{'fact': 'The procedure involves going to the...,0.042710,...,0.000000,0.000000,[{'fact': 'The procedure involves going to the...,[{'fact': 'Use the same import parameters as i...,1122,2,1124,7442,1835,9277
1,PS-8,Customer reports that the document is stuck in...,The error message 'HTTP response '401: Unautho...,None,None,None,None,None,[{'fact': 'The document is stuck in 'Received ...,-0.049147,...,0.555556,0.600000,[{'fact': 'The document is stuck in 'Received ...,[{'fact': 'The error message is 'HTTP response...,1369,2,1371,9530,1148,10678
2,EA-41,"In DRC extensibility, what are the supported c...",The supported countres for external documents ...,None,None,None,None,None,[{'fact': 'Malaysia is a supported country for...,-0.198704,...,0.000000,0.000000,[{'fact': 'Malaysia is a supported country for...,[],905,2,907,14034,1693,15727
3,PS-5,We as Syntax are currently implementing SAP DR...,"1. In the BTP Subaccount, go to Connectivity -...",None,None,None,None,None,"[{'fact': 'In the BTP Subaccount, go to Connec...",0.037810,...,0.000000,0.100000,"[{'fact': 'File Extension should be 'PFX'.', '...","[{'fact': 'In the BTP Subaccount, go to Connec...",1127,2,1129,15123,1779,16902
4,PS-3,Incoming ZUGFeRD invoices (from suppliers) we ...,Here are some hints that might be helpful. We ...,None,None,None,None,None,[{'fact': 'Extend EDOPROCFUNCASGV as indicated...,-0.018870,...,0.000000,0.000000,[{'fact': 'Extend EDOPROCFUNCASGV as indicated...,[{'fact': 'EDOACTIONUIPROCV is indicated in Co...,1074,2,1076,14101,1718,15819
5,EA-5,"As a Public Cloud customer in Italy, can I ext...","Yes, as a Public Cloud customer in Italy, you...",None,None,None,None,None,[{'fact': 'As a Public Cloud customer in Italy...,-0.016739,...,0.333333,0.500000,[{'fact': 'As a Public Cloud customer in Italy...,[{'fact': 'You can define that for some types ...,933,2,935,13846,1617,15463
6,EA-9,Can I extend the Manage Electronic Documents a...,"Yes, you can extend the Manage Electronic Docu...",None,None,None,None,None,[{'fact': 'You can extend the Manage Electroni...,0.192230,...,1.000000,1.000000,[{'fact': 'You can extend the Manage Electroni...,[{'fact': 'Extension can be done using Busines...,888,2,890,8053,1639,9692
7,EA-52,Can I control the visibility of Electronic Doc...,Yes you restrict the visibility of electronic ...,None,None,None,None,None,[{'fact': 'You can restrict the visibility of ...,-0.087736,...,0.000000,0.333333,[{'fact': 'You can restrict the visibility of ...,[],1003,2,1005,13540,1752,15292
8,EA-40,I am a Public Cloud customer researching the s...,Source documents that have the same eDocument ...,None,None,None,None,None,[{'fact': 'Source documents with the same eDoc...,-0.250997,...,0.000000,0.000000,[{'fact': 'Source documents with the same eDoc...,[{'fact': 'Regular invoices and their correcti...,513,2,515,8714,1237,9951
9,EA-45,Before implementing a new process in DRC using...,"Before you implement a new process, there are ...",None,None,None,None,None,[{'fact': 'Determine which new actions the eDo...,0.021719,...,0.000000,0.000000,[{'fact': 'Determine which new actions the eDo...,[],886,2,888,3763,498,4261


In [8]:
# Convert to list of dictionaries (JSON objects)
json_list = df.to_dict(orient="records")

with open(result_file_name, "w") as f:
    json.dump(json_list, f, indent=2)